<a href="https://colab.research.google.com/github/Morningspread/Angry-Tweeter/blob/main/Inference_Twit_COVID_223K_March15%2C2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Running our inference pipeline on Twitter data... 

Dataset: 

Twint COVID-19 scrape - 223K tweets... 

Path to models...


1.   /content/drive/MyDrive/Sean/Emoclass/Model_Anger_10words_70acc_Feb25,2023
2.   /content/drive/MyDrive/Sean/Emoclass/Model_Anger_75words_71acc_Feb28,2023
3. /content/drive/MyDrive/Sean/Emoclass/Model_Fear_151K_11words_75acc_Feb24,2023
4. /content/drive/MyDrive/Sean/Emoclass/Model_Fear_78acc_75words_Feb17b,2022

Requirements
All the models need to be trained on the same input length... 






In [30]:
# Load the Drive helper and mount

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
#This is supposed to help us navigate to our folder... 

import os
os.chdir('/content/drive/MyDrive/Sean/Emoclass')

In [32]:
#Loading the dependencies.. 

%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re 

#Loading the tokenizers...

from tensorflow.keras.preprocessing.text import Tokenizer

#Importing the pad sequence... 
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Loading the training models and datasets 

In [33]:
# Load trained models - only need to do this once per run... 

#anger_model_10 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Anger_10words_70acc_Feb25,2023')
#anger_model_75 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Anger_75words_71acc_Feb28,2023')
#fear_model_10 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Fear_151K_11words_75acc_Feb24,2023')
#fear_model_75 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Fear_78acc_75words_Feb17b,2022')


In [34]:
# Load inference dataset... 

df = pd.read_csv("/content/drive/MyDrive/Sean/Twitter/COVID_223KTweets.csv")

In [35]:
df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [36]:
# rename the 'tweet' column to 'text'
df = df.rename(columns={'tweet': 'text'})

In [37]:
#Checking for missing variables...

df.isna().sum()

id                      0
conversation_id         0
created_at              0
date                    0
time                    0
timezone                0
user_id                 0
username                0
name                    9
place              223844
text                    0
language                0
mentions                0
urls                    0
photos                  0
replies_count           0
retweets_count          0
likes_count             0
hashtags                0
cashtags                0
link                    0
retweet                 0
quote_url          206532
video                   0
thumbnail          163436
near               223955
geo                223955
source             223955
user_rt_id         223955
user_rt            223955
retweet_id         223955
reply_to                0
retweet_date       223955
translate          223955
trans_src          223955
trans_dest         223955
dtype: int64

##Filtering the non-english tweets - 116K english... 

Check to see if index needs to be reset...

In [38]:
#We need to reset the index here...and drop the old one. 

df = df[df['language'] == 'en'].reset_index(drop=True)

In [39]:
#Let's check and see what's happening with the dataframe... 

df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1578109493392515073,1578109493392515073,2022-10-06 19:46:37 UTC,2022-10-06,19:46:37,0,1518980837135589376,krystynaklepin1,Christine K.,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1578109461184454656,1578101418317082624,2022-10-06 19:46:30 UTC,2022-10-06,19:46:30,0,1497906018029907970,turdfur58439807,BlackSanta9111,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'montanamike_tx', 'name': 'mi...",NaN,NaN,NaN,NaN
2,1578109438464241666,1578087822858522630,2022-10-06 19:46:24 UTC,2022-10-06,19:46:24,0,16635842,spectrummag,SPECTRUM (Since 1969),NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1578109398215426048,1578109398215426048,2022-10-06 19:46:15 UTC,2022-10-06,19:46:15,0,36803382,yakimahealth,Yakima Health District,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1578109382625067008,1578109382625067008,2022-10-06 19:46:11 UTC,2022-10-06,19:46:11,0,1550501740956422147,smslausd,Student Medical Services & Medi-Cal Programs L...,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


##Data cleaning step... 

We want to maintain the original.... text in the Tweet so that it doesn't interfere with the pre-processing.

In [41]:
# select only the 'text' column for cleaning, tokenization and inference... 

text = df['text']

In [ ]:
import re

# define a function to remove @handles, URLs, hashtags, and apostrophes from a string
def preprocess_text(text):
    text = re.sub(r'@\w+\s?', '', text) # remove @handles
    text = re.sub(r'http\S+', '', text) # remove URLs
    text = re.sub(r'#\w+\s?', '', text) # remove hashtags
    text = text.replace("'", "") # remove apostrophes
    text = text.replace("?", "") # remove question marks
    text = text.replace("!", "") # remove exclamation marks
    return text

# apply the function to the 'text' variable
text = preprocess_text(text)


#Tokenizing and Padding 

In [42]:
# Define functions for tokenizing and padding
def tokenize_and_pad_sequences(text, tokenizer, max_len):
    sequences = tokenizer.texts_to_sequences(text)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len, padding='post')
    return padded_sequences


In [43]:
# Tokenize and pad input sequences
tokenizer = Tokenizer(num_words=93886, oov_token='<UNK>')
tokenizer.fit_on_texts(text)
padded_sequences_400 = tokenize_and_pad_sequences(text, tokenizer, 400)
padded_sequences_300 = tokenize_and_pad_sequences(text, tokenizer, 300)

# Generate the predictions
anger_10_preds = np.argmax(anger_model_10.predict(padded_sequences_400), axis=-1)
anger_75_preds = np.argmax(anger_model_75.predict(padded_sequences_400), axis=-1)
fear_10_preds = np.argmax(fear_model_10.predict(padded_sequences_300), axis=-1)
fear_75_preds = np.argmax(fear_model_75.predict(padded_sequences_400), axis=-1)

# Create dataframe of predictions
pred_df = pd.DataFrame({
    'text': text,
    'anger_10': anger_10_preds,
    'anger_75': anger_75_preds,
    'fear_10': fear_10_preds,
    'fear_75': fear_75_preds
})


3647/3647 [==============================] - 144s 39ms/step


#Defining the labels for the emotions... 

In [44]:
# Define emotion labels
anger_labels = ['calm', 'annoyed', 'agitated', 'frustrated', 'angry', 'rage']
fear_labels = ['calm', 'concern', 'worry', 'anxiety', 'fear', 'dread']

# Define mapping from integer label to emotion label
anger_label_map = {i: label for i, label in enumerate(anger_labels)}
fear_label_map = {i: label for i, label in enumerate(fear_labels)}

# Add columns for predicted emotion label
pred_df['anger_10_label'] = pred_df['anger_10'].map(anger_label_map)
pred_df['anger_75_label'] = pred_df['anger_75'].map(anger_label_map)
pred_df['fear_10_label'] = pred_df['fear_10'].map(fear_label_map)
pred_df['fear_75_label'] = pred_df['fear_75'].map(fear_label_map)


##Creating sum variables... 

In [45]:
pred_df['anger_total'] = pred_df['anger_10'] + pred_df['anger_75']
pred_df['fear_total'] = pred_df['fear_10'] + pred_df['fear_75']
pred_df['emotion_total'] = pred_df['anger_total'] + pred_df['fear_total']

#Tying to Merge Pred_df and original dataframe... 

In [46]:
pred_df.head()

,text,anger_10,anger_75,fear_10,fear_75,anger_10_label,anger_75_label,fear_10_label,fear_75_label,anger_total,fear_total,emotion_total
0,SAFE &amp; EFFECTIVE. Pfizer skipped major cat...,1,2,5,4,annoyed,agitated,dread,fear,3,9,12
1,Its a very small cohort but it represented unv...,4,4,4,3,angry,angry,fear,anxiety,8,7,15
2,"Lisa Beardsley-Hardy, GC director of education...",4,1,4,0,angry,annoyed,fear,calm,5,4,9
3,West Valley Junior High School will hold a com...,4,2,1,0,angry,agitated,concern,calm,6,1,7
4,Free Covid-19 vaccinations will be provided by...,5,4,1,0,rage,angry,concern,calm,9,1,10


In [51]:
#Merging on index... 

merged = df.merge(pred_df, left_index=True, right_index=True, how='inner', indicator=True)
value_counts = merged['_merge'].value_counts()
print(value_counts)


both          116687
left_only          0
right_only         0
Name: _merge, dtype: int64


In [ ]:
merged.head()


In [54]:
merged.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'text_x', 'language',
       'mentions', 'urls', 'photos', 'replies_count', 'retweets_count',
       'likes_count', 'hashtags', 'cashtags', 'link', 'retweet', 'quote_url',
       'video', 'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest', 'text_y', 'anger_10', 'anger_75', 'fear_10', 'fear_75',
       'anger_10_label', 'anger_75_label', 'fear_10_label', 'fear_75_label',
       'anger_total', 'fear_total', 'emotion_total', '_merge'],
      dtype='object')

In [56]:
#Createa a dataframe to check the two...
new_df = merged[['text_x', 'text_y']]

In [57]:
new_df

,text_x,text_y
0,SAFE &amp; EFFECTIVE. Pfizer skipped major cat...,SAFE &amp; EFFECTIVE. Pfizer skipped major cat...
1,Its a very small cohort but it represented unv...,Its a very small cohort but it represented unv...
2,"Lisa Beardsley-Hardy, GC director of education...","Lisa Beardsley-Hardy, GC director of education..."
3,West Valley Junior High School will hold a com...,West Valley Junior High School will hold a com...
4,Free Covid-19 vaccinations will be provided by...,Free Covid-19 vaccinations will be provided by...
...,...,...
116682,""" Safe and Effective: A Second Opinion shines ...",""" Safe and Effective: A Second Opinion shines ..."
116683,Covid-19 inspired people to circumvent censors...,Covid-19 inspired people to circumvent censors...
116684,Just like trump caused Covid-19 right,Just like trump caused Covid-19 right
116685,Hastings restaurant owner said she was fined b...,Hastings restaurant owner said she was fined b...


#Exporting the dataset - if required... 

In [47]:
#path = "/content/drive/MyDrive/Sean/Emoclass/inference_COVID19_223K_March9b,2023.csv"
#pred_df.to_csv(path)